# RL basics

Термины и понятия:

- агент/среда
- наблюдение $o$ / состояние $s$
- действие $a$, стратегия $\pi: \pi(s) \rightarrow a$ функция перехода $T: T(s, a) \rightarrow s'$
- вознаграждение $r$, ф-я вознаграждений $R: R(s, a) \rightarrow r$
- цикл взаимодействия, траектория $\tau: (s_0, a_0, r_0, s_1, a_1, r_1, ..., s_T, a_T, r_T)$, эпизод
- отдача $G$, подсчет отдачи, средняя[/ожидаемая] отдача $\mathbb{E}[G]$

In [57]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [58]:
import glob
import io
import base64
import gymnasium as gym
import numpy as np
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

## Agent, environment

<img src=https://gymnasium.farama.org/_images/lunar_lander.gif caption="lunar lander" width="150" height="50"><img src=https://gymnasium.farama.org/_images/mountain_car.gif caption="mountain car" width="150" height="50">
<img src=https://gymnasium.farama.org/_images/cliff_walking.gif caption="cliff walking" width="300" height="50">
<img src=https://ale.farama.org/_images/montezuma_revenge.gif caption="montezuma revenge" width="150" height="100">
<img src=https://github.com/danijar/crafter/raw/main/media/video.gif caption="crafter" width="150" height="100">
<img src=https://camo.githubusercontent.com/6df2ca438d8fe8aa7a132b859315147818c54af608f8609320c3c20e938acf48/68747470733a2f2f6d656469612e67697068792e636f6d2f6d656469612f344e78376759694d394e44724d724d616f372f67697068792e676966 caption="malmo minecraft" width="150" height="100">
<img src=https://images.ctfassets.net/kftzwdyauwt9/e0c0947f-1a44-4528-4a41450a9f0a/2d0e85871d58d02dbe01b2469d693d4a/table-03.gif caption="roboschool" width="150" height="100">
<img src=https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/02/mdp.png caption="Марковский процесс принятия решений" width="150" height="100">
<img src=https://minigrid.farama.org/_images/DoorKeyEnv.gif caption="minigrid" width="120" height="120">

## Observation, state

TODO:
- добавить примеры наблюдений/состояний (числа, векторы, картинки)
- интуитивное объяснение различия, положить пока, что наблюдение = состояние
- пространство состояний


В каждый момент времени среда имеет некоторое внутреннее состояние. Здесь слово "состояние" я употребил скорее в интуитивном понимании, чтобы обозначить, что среда изменчива (иначе какой смысл с ней взаимодействовать, если ничего не меняется). В обучении с подкреплением под термином состояние $s$ (или $s_t$, где $t$ — текущее время) подразумевают либо абстрактно информацию о "состоянии" среды, либо ее явное представление в виде данных, достаточные для полного описания "состояния". *NB: Здесь можно провести аналогию с компьютерными играми — файл сохранения игры как раз содержит информацию о "состоянии" мира игры, чтобы в будущем можно было продолжить с текущей точки, так что данные этого файла в целом можно с некоторой натяжкой считать состоянием (с натяжкой, потому что редко когда в сложных играх файлы сохранения содержат прямо вот всю информацию, так что после перезагрузки вы получите не совсем точную копию). При этом обычно подразумевается, что состояние не содержит в себе ничего лишнего, то есть это **минимальный** набор информации.*

Наблюдением $o$ называют то, что агент "видит" о текущем состоянии среды. Это не обязательно зрение, а вообще вся доступная ему информация (условно, со всех его органов чувств).

В общем случае наблюдение: кортеж/словарь многомерных векторов чисел.

In [59]:
print(gym.make("CartPole-v0").reset()[0].shape)
print(gym.make("MountainCar-v0").reset()[0].shape)

(4,)
(2,)


/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


## Action, policy, transition function

Рассмотрим следующие MDP:

- A: <img src=https://i.ibb.co/mrCMVZLQ/mdp-a.png caption="A" width="400" height="100">
- B: <img src=https://i.ibb.co/GQ2tVtjC/mdp-b.png caption="B" width="400" height="100">

Links to all:
[A](https://i.ibb.co/mrCMVZLQ/mdp-a.png)
[B](https://i.ibb.co/GQ2tVtjC/mdp-b.png)
[C](https://i.ibb.co/Jj9LYHjP/mdp-c.png)
[D](https://i.ibb.co/Y47Mr83b/mdp-d.png)
[E](https://i.ibb.co/Kjt1Xhmf/mdp-e.png)

Давайте явно запишем пространства состояний $S$ и действий $A$, а также функцию перехода $T$ среды.

In [60]:
states = set(range(3))
actions = set(range(1))

print(f'{states=} | {actions=}')

T = {
    (0, 0): 1,
    (1, 0): 2,
    (2, 0): 2
}
print(f'Transition function {T=}')

A_mdp = states, actions, T

states={0, 1, 2} | actions={0}
Transition function T={(0, 0): 1, (1, 0): 2, (2, 0): 2}


Попробуйте записать функцию перехода в матричном виде:

In [61]:
p1, p2, p3 = 0.5, 0.3, 0.2

P = np.array([
    [0, p1, p2, p3],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

print("Матрица переходов P:")
print(P)


Матрица переходов P:
[[0.  0.5 0.3 0.2]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]


Как получить вероятность нахождения агента в состоянии (1) через N шагов? Что происходит с вероятностями нахождения в состояниях при $N \rightarrow \infty$

In [62]:

def probability_after_n_steps(initial_dist, P, n, target_state):
    P_n = np.linalg.matrix_power(P, n)
    result_dist = initial_dist @ P_n
    return result_dist[target_state]

initial_dist = np.array([1.0, 0.0, 0.0, 0.0])
n = 5
target = 1

prob = probability_after_n_steps(initial_dist, P, n, target)
print(f"\nВероятность в состоянии {target} через {n} шагов: {prob}")

def stationary_distribution(P, tolerance=1e-10, max_iter=1000):
    pi = np.ones(P.shape[0]) / P.shape[0]
    for i in range(max_iter):
        pi_next = pi @ P
        if np.max(np.abs(pi_next - pi)) < tolerance:
            break
        pi = pi_next
    return pi

stationary = stationary_distribution(P)
print(f"\nСтационарное распределение: {stationary}")


Вероятность в состоянии 1 через 5 шагов: 0.5

Стационарное распределение: [0.    0.375 0.325 0.3  ]


Задайте еще несколько MDP:

- C: <img src=https://i.ibb.co/Jj9LYHjP/mdp-c.png caption="C" width="400" height="100">

In [63]:
p0, p1 = 0.7, 0.3

P = np.array([
    [0, p0, p1, 0],
    [0, p0, 0, p1],
    [0, 0, p1, p0],
    [0, 0, 0, 1]
])

print("Матрица переходов P:")
print(P)

def probability_after_n_steps(initial_dist, P, n, target_state):
    P_n = np.linalg.matrix_power(P, n)
    result_dist = initial_dist @ P_n
    return result_dist[target_state]

initial_dist = np.array([1.0, 0.0, 0.0, 0.0])
n = 5
target = 3

prob = probability_after_n_steps(initial_dist, P, n, target)
print(f"\nВероятность в состоянии {target} через {n} шагов: {prob}")

def stationary_distribution(P, tolerance=1e-10, max_iter=1000):
    pi = np.ones(P.shape[0]) / P.shape[0]
    for i in range(max_iter):
        pi_next = pi @ P
        if np.max(np.abs(pi_next - pi)) < tolerance:
            break
        pi = pi_next
    return pi

stationary = stationary_distribution(P)
print(f"\nСтационарное распределение: {stationary}")

Матрица переходов P:
[[0.  0.7 0.3 0. ]
 [0.  0.7 0.  0.3]
 [0.  0.  0.3 0.7]
 [0.  0.  0.  1. ]]

Вероятность в состоянии 3 через 5 шагов: 0.8295

Стационарное распределение: [0.00000000e+00 2.54010930e-10 2.11955791e-32 1.00000000e+00]


Давайте попробуем задать двух агентов: случайного и оптимального (для каждой среды свой).

In [64]:
class Agent:
    def __init__(self, actions):
        self.rng = np.random.default_rng()
        self.actions = np.array(list(actions))

    def act(self, state):
        return self.rng.integers(len(self.actions))

В качестве дополнения, запишите стратегию агента

In [65]:
class AgentA:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        return 0

## Reward, reward function

Теперь добавим произвольную функцию вознаграждения. Например, для A:

In [66]:
states_A = set(range(3))
actions_A = set(range(1))

T_A = {
    (0, 0): 1,
    (1, 0): 2,
    (2, 0): 2
}

R_A = {
    (0, 0): -0.1,
    (1, 0): 1.0,
    (2, 0): 0.0
}

print(f'{states_A=} | {actions_A=}')
print(f'Transition function {T_A=}')
print(f'Reward function {R_A=}')

terminal_states_A = {2}
A_mdp = states_A, actions_A, T_A, R_A, terminal_states_A
print(f'A_mdp = {A_mdp}')

states_A={0, 1, 2} | actions_A={0}
Transition function T_A={(0, 0): 1, (1, 0): 2, (2, 0): 2}
Reward function R_A={(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0}
A_mdp = ({0, 1, 2}, {0}, {(0, 0): 1, (1, 0): 2, (2, 0): 2}, {(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0}, {2})


## Interaction loop, trajectory, termination, truncation, episode

Общий цикл взаимодействия в рамках эпизода:
1. Инициализировать среду: $s \leftarrow \text{env.init()}$
2. Цикл:
    - выбрать действие: $a \leftarrow \pi(s)$
    - получить ответ от среды: $s, r, d \leftarrow \text{env.next(a)}$
    - если $d == \text{True}$, выйти из цикла

In [67]:
def run_episode(mdp, agent_class, max_steps=100):
    states, actions, T, R, terminal_states = mdp
    agent = agent_class(actions)

    s = 0
    tau = []
    for step in range(max_steps):
        a = agent.act(s)
        s_next = T[(s, a)]
        r = R[(s, a)]

        tau.append((s, a, r))
        s = s_next

        if s in terminal_states:
            break

    return tau

In [68]:
run_episode(A_mdp,AgentA)

[(0, 0, -0.1), (1, 0, 1.0)]

Termination — означает окончание эпизода, когда достигнуто терминальное состояние. Является частью задания среды.

Truncation — означает окончание эпизода, когда достигнут лимит по числу шагов (=времени). Обычно является внешне заданным параметром для удобства обучения.

Пока не будем вводить truncation, но поддержим termination: расширьте определение среды информацией о терминальных состояниях для всех описанных ранее сред. Сгенерируйте по несколько случайных траекторий для каждой среды.

### Return, expected return

Наиболее важная метрика оценки качества работы агента: отдача.

Отдача: $G(s_t) = \sum_{i=t}^T r_i$

Обычно также вводят параметр $\gamma \in [0, 1]$, дисконтирующий будущие вознаграждения. А еще, тк отдача может меняться от запуска к запуску благодаря вероятностным процессам, нас интересует отдача в среднем — ожидаемая отдача:

$$\hat{G}(s_t) = \mathbb{E} [ \sum_{i=t}^T \gamma^{i-t} r_i ]$$

Именно ее и оптимизируют в RL.

Давайте научимся считать отдачу для состояний по траектории и считать среднюю отдачу.

In [69]:
def calculate_returns(trajectory, gamma=1.0):
    returns = []
    for i in range(len(trajectory)):
        G = sum(r * (gamma ** (j-i)) for j, (_, _, r) in enumerate(trajectory[i:]))
        returns.append(G)
    return returns

def calculate_expected_return(mdp, agent_class, num_episodes=100, gamma=1.0):
    returns_list = []

    for i in range(num_episodes):
        trajectory = run_episode(mdp, agent_class)
        if trajectory:
            G = sum(r for _, _, r in trajectory)
            returns_list.append(G)

    mean_return = np.mean(returns_list)
    return mean_return, returns_list

In [70]:
trajectory = run_episode(A_mdp, AgentA)
print("Trajectory:", trajectory)

returns = calculate_returns(trajectory)
print("Returns:", returns)

mean_return, all_returns = calculate_expected_return(A_mdp, AgentA)
print("Mean return:", mean_return)

Trajectory: [(0, 0, -0.1), (1, 0, 1.0)]
Returns: [0.9, 1.0]
Mean return: 0.9000000000000005


Для среды С


In [71]:
states_C = set(range(4))
actions_C = set(range(2))

T_C = {
    (0, 0): 1, (0, 1): 2,
    (1, 0): 1, (1, 1): 3,
    (2, 0): 2, (2, 1): 3,
    (3, 0): 3, (3, 1): 3
}

R_C = {
    (0, 0): -0.5, (0, 1): -0.2,
    (1, 0): -0.3, (1, 1): 1.5,
    (2, 0): -0.3, (2, 1): 0.8,
    (3, 0): 0.0, (3, 1): 0.0
}

terminal_states_C = {3}

print(f'{states_C=} | {actions_C=}')
print(f'Transition function {T_C=}')
print(f'Reward function {R_C=}')

C_mdp = states_C, actions_C, T_C, R_C, terminal_states_C
print(f'C_mdp = {C_mdp}')

states_C={0, 1, 2, 3} | actions_C={0, 1}
Transition function T_C={(0, 0): 1, (0, 1): 2, (1, 0): 1, (1, 1): 3, (2, 0): 2, (2, 1): 3, (3, 0): 3, (3, 1): 3}
Reward function R_C={(0, 0): -0.5, (0, 1): -0.2, (1, 0): -0.3, (1, 1): 1.5, (2, 0): -0.3, (2, 1): 0.8, (3, 0): 0.0, (3, 1): 0.0}
C_mdp = ({0, 1, 2, 3}, {0, 1}, {(0, 0): 1, (0, 1): 2, (1, 0): 1, (1, 1): 3, (2, 0): 2, (2, 1): 3, (3, 0): 3, (3, 1): 3}, {(0, 0): -0.5, (0, 1): -0.2, (1, 0): -0.3, (1, 1): 1.5, (2, 0): -0.3, (2, 1): 0.8, (3, 0): 0.0, (3, 1): 0.0}, {3})


In [72]:
class OptimalAgentC:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        if state == 0:
            return 0
        elif state == 1:
            return 1
        elif state == 2:
            return 1
        else:
            return 0

In [73]:
random_agent = Agent(actions_C)
optimal_agent = OptimalAgentC(actions_C)

print("Random agent trajectories:")
for i in range(3):
    trajectory = run_episode(C_mdp, Agent)
    returns = calculate_returns(trajectory)
    print(f"Episode {i+1}: {trajectory}")
    print(f"Returns: {returns}")

print("\nOptimal agent trajectories:")
for i in range(3):
    trajectory = run_episode(C_mdp, OptimalAgentC)
    returns = calculate_returns(trajectory)
    print(f"Episode {i+1}: {trajectory}")
    print(f"Returns: {returns}")

random_mean, random_returns = calculate_expected_return(C_mdp, Agent, 1000)
optimal_mean, optimal_returns = calculate_expected_return(C_mdp, OptimalAgentC, 1000)

print(f"\nRandom agent mean return: {random_mean:.3f}")
print(f"Optimal agent mean return: {optimal_mean:.3f}")

Random agent trajectories:
Episode 1: [(0, np.int64(1), -0.2), (2, np.int64(1), 0.8)]
Returns: [0.6000000000000001, 0.8]
Episode 2: [(0, np.int64(1), -0.2), (2, np.int64(1), 0.8)]
Returns: [0.6000000000000001, 0.8]
Episode 3: [(0, np.int64(1), -0.2), (2, np.int64(1), 0.8)]
Returns: [0.6000000000000001, 0.8]

Optimal agent trajectories:
Episode 1: [(0, 0, -0.5), (1, 1, 1.5)]
Returns: [1.0, 1.5]
Episode 2: [(0, 0, -0.5), (1, 1, 1.5)]
Returns: [1.0, 1.5]
Episode 3: [(0, 0, -0.5), (1, 1, 1.5)]
Returns: [1.0, 1.5]

Random agent mean return: 0.478
Optimal agent mean return: 1.000
